In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
import glob
path_dataset_1=glob.glob("../input/dataset-1/train_png/*.png")
path_dataset_2=glob.glob("../input/dataset-2/train_png/*.png")
path_dataset_3=glob.glob("../input/dataset-3/train_png/*.png")
path_dataset_4_1=glob.glob("../input/dataset-4-1/train_png/*.png")
path_dataset_4_1.remove('../input/dataset-4-1/train_png/79e30d_0.png')
path_dataset_4_2=glob.glob("../input/dataset-4-2/train_png/*.png")
path_dataset_5_1=glob.glob("../input/dataset-5-1-index-yok/train_png/*.png")
path_dataset_5_1.remove('../input/dataset-5-1-index-yok/train_png/d7a20d_0.png')
path_dataset_5_2=glob.glob("../input/dataset-5-2/train_png/*.png")

path_dataset_5_2.remove('../input/dataset-5-2/train_png/f9fc6b_0.png')


In [15]:
train_png_paths=path_dataset_1+path_dataset_2+path_dataset_3+path_dataset_4_1+path_dataset_4_2+path_dataset_5_1+path_dataset_5_2
len(train_png_paths)

518

In [16]:
import numpy as np
DATASET_FOLDER = "/kaggle/input/mayo-clinic-strip-ai/"


path_csv = os.path.join(DATASET_FOLDER, "train.csv")
df_train = pd.read_csv(path_csv)
list_df=[]
for i in df_train.index:
    for ii in range(len(train_png_paths)):
        if df_train['image_id'][i]==train_png_paths[ii].split('/')[-1][:-4]:
            list_df.append([train_png_paths[ii],df_train['label'][i]])

In [17]:
df=pd.DataFrame(list_df,columns=['image_path','label'])
df

,image_path,label
0,../input/dataset-2/train_png/008e5c_0.png,CE
1,../input/dataset-1/train_png/026c97_0.png,CE
2,../input/dataset-2/train_png/032f10_0.png,CE
3,../input/dataset-1/train_png/0372b0_0.png,CE
4,../input/dataset-4-1/train_png/03d1ec_0.png,LAA
...,...,...
513,../input/dataset-2/train_png/fde9ba_0.png,LAA
514,../input/dataset-2/train_png/fe0cca_0.png,CE
515,../input/dataset-2/train_png/fe9645_0.png,CE
516,../input/dataset-3/train_png/ff14e0_0.png,CE


In [18]:
df_copy=df.copy()
Fseries = pd.Series(df_copy.image_path, name="filepaths")

Lseries = pd.Series(df_copy.label, name="labels")
inme_data = pd.concat([Fseries,Lseries], axis=1)
inmer_df = pd.DataFrame(inme_data)
print(inmer_df.head())
print(inmer_df["labels"].value_counts())

#shape of datatset
inmer_df.shape

                                     filepaths labels
0    ../input/dataset-2/train_png/008e5c_0.png     CE
1    ../input/dataset-1/train_png/026c97_0.png     CE
2    ../input/dataset-2/train_png/032f10_0.png     CE
3    ../input/dataset-1/train_png/0372b0_0.png     CE
4  ../input/dataset-4-1/train_png/03d1ec_0.png    LAA
CE     392
LAA    126
Name: labels, dtype: int64


(518, 2)

In [19]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report
#tensorflow libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D, Dense, Dropout,Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

from tensorflow.keras.callbacks import TensorBoard,EarlyStopping 

In [25]:
image_gen = ImageDataGenerator(preprocessing_function= tf.keras.applications.mobilenet_v2.preprocess_input)
train = image_gen.flow_from_dataframe(dataframe= inmer_df,x_col="filepaths",y_col="labels",
                                      target_size=(128,128),

                                      class_mode='categorical', #used for Sequential Model
                                      batch_size=2,
                                      shuffle=False            #do not shuffle data
                                     )

Found 518 validated image filenames belonging to 2 classes.


In [26]:
## from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
base_model = InceptionV3(input_shape = (128, 128, 3), include_top = False, weights = 'imagenet')
for layer in base_model.layers:
    layer.trainable = False
from tensorflow.keras.optimizers import RMSprop

x = keras.layers.Flatten()(base_model.output)
x = keras.layers.Dense(1024, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)

# Add a final sigmoid layer with 1 node for classification output
x = keras.layers.Dense(2, activation='sigmoid')(x)

model = tf.keras.models.Model(base_model.input, x)

model.compile(optimizer = RMSprop(lr=0.0001), loss = 'binary_crossentropy', metrics = ['acc'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint,TensorBoard,LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
earlystopping = EarlyStopping(monitor='val_loss',
                              mode='min', 
                              verbose=1, 
                              patience=30
                             )
# save the best model with lower validation loss
checkpointer = ModelCheckpoint(filepath="./class_model_EfficientNetB7.h5", 
                               verbose=1, 
                               save_best_only=True
                              )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              mode='min',
                              verbose=1,
                              patience=10,
                              min_delta=0.0001,
                              factor=0.2
                             )

In [27]:
inc_history = model.fit_generator(train, epochs = 20,
                                        callbacks = [checkpointer, earlystopping, reduce_lr])


Epoch 1/3
100/100 [==============================] - 2630s 26s/step - loss: 1.4539 - acc: 0.6750
Epoch 2/3
  6/100 [>.............................] - ETA: 32:07 - loss: 1.0665 - acc: 0.7500

KeyboardInterrupt: 